<a href="https://colab.research.google.com/github/pranarahayu/Football-Data/blob/main/Dashboard_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import glob
from datetime import date
import numpy as np

In [ ]:
#mount to gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

path = '/content/gdrive/MyDrive/Liga Indonesia 2022/Matches'
all_files = glob.glob(os.path.join(path, "*.xlsx"))

df = pd.concat((pd.read_excel(f, skiprows=[0]) for f in all_files), ignore_index=True)

from datetime import date
date = date.today().strftime("%Y-%m-%d")
filename = 'sum_ligaId_upd' + date +'.xlsx'

df.to_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Summary/'+filename, index=False)

Mounted at /content/gdrive


In [2]:
#mount to gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

path2 = '/content/gdrive/MyDrive/Liga Indonesia 2022/Summary'
files = glob.glob(os.path.join(path2, "*.xlsx"))

df = pd.DataFrame()
newest = max(files, key=os.path.getctime)
data = pd.read_excel(newest)
df = df.append(data)

df.loc[((df['Team'] == 'Bali United FC') & (df['Name'] == 'Muhammad Ridho')), 'Name'] = 'Muhammad Ridho Djazulie'
df.loc[((df['Team'] == 'PSS Sleman') & (df['Name'] == 'Muhammad Ridwan')), 'Name'] = 'Muhammad Ridwan (Sleman)'
df.loc[((df['Team'] == 'PERSEKAT Kab. Tegal') & (df['Name'] == 'Muhammad Ridwan')), 'Name'] = 'Muhammad Ridwan (Tegal)'
df.loc[((df['Team'] == 'Gresik United') & (df['Name'] == 'Muhammad Fahri')), 'Name'] = 'Muhammad Fahri (Gresik)'
df.loc[((df['Team'] == 'Nusantara United FC') & (df['Name'] == 'Muhammad Fahri')), 'Name'] = 'Muhammad Fahri (NUFC)'
df.loc[((df['Team'] == 'PSIS Semarang') & (df['Name'] == 'Taufik Hidayat')), 'Name'] = 'Taufik Hidayat (Smg)'
df.loc[((df['Team'] == 'PERSIJA Jakarta') & (df['Name'] == 'Taufik Hidayat')), 'Name'] = 'Taufik Hidayat (Jkt)'

df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df['Name'] = df['Name'].str.replace('  ', ' ')


df['Name'] = df['Name'].replace(['Daisuke Caumanday Sato','Arodi Uopdana','Tomi Darmawan','Rachmat Latief'],['Daisuke Sato','Arody Uopdana','Tommy Darmawan','Rachmat Latif'])

player_db = pd.read_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Database/database_pemain.xlsx')
club_db = pd.read_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Database/database_klub.xlsx')

player_db_obj = player_db.select_dtypes(['object'])
player_db[player_db_obj.columns] = player_db_obj.apply(lambda x: x.str.strip())
player_db['Name'] = player_db['Name'].str.replace('  ', ' ')

df1 = pd.merge(df,player_db,on='Name',how='left')
df2 = pd.merge(df1,club_db,on='Team',how='left')

Mounted at /content/gdrive


In [ ]:
df3 = df2[df2['Kompetisi'].str.contains("Liga 2")]
df3 = df3.drop(['No','Gameweek','Opponent','Position (in match)','Match','Home/Away','Venue','Date','Result','Starter/Subs','Subs','Ball Possession','DoB','Kompetisi','Position','Nationality','Pref. Foot','badge'], axis=1)

df_sum = df3.groupby(['Name','Team'], as_index=False).sum()

df_data = df_sum[['Name','Team','MoP','Goal','Shot on','Shot off','Shot Blocked','Create Chance','Assist','Pass','Pass Fail','Cross','Dribble','Tackle','Intercept','Intercept Fail','Recovery','Block','Aerial Won','Aerial Lost','Save','Shots on Target Faced','Penalty Save','Concede Penalty','Keeper - Sweeper', 'Cross Claim','Error Goal', 'Goal Kick', 'Goal Kick - Goal Kick Launch', 'Penalty Goal', 'Pass - Progressive Pass', 'Pass - Through Pass']]

df_data['Shots'] = df_data['Shot on'] + df_data['Shot off'] + df_data['Shot Blocked']
df_data['Aerial Duels'] = df_data['Aerial Won'] + df_data['Aerial Lost']
df_data['Passes'] = df_data['Pass'] + df_data['Pass Fail']
df_data['Intercepts'] = df_data['Intercept'] + df_data['Intercept Fail']
df_data['Saves'] = df_data['Save'] + df_data['Penalty Save']
df_data['Goals'] = df_data['Goal'] + df_data['Penalty Goal']

##########
data_fix = df_data[['Name','Team','MoP','Goals','Shots','Shot on','Create Chance','Assist','Pass','Passes','Pass - Progressive Pass', 'Pass - Through Pass','Cross','Dribble','Tackle','Intercepts','Recovery','Block','Aerial Duels','Aerial Won','Saves','Shots on Target Faced','Penalty Save','Keeper - Sweeper','Cross Claim','Goal Kick','Goal Kick - Goal Kick Launch']]

data_fix = data_fix.drop(['Name','Team'], axis=1)

def p90_Calculator(variable_value, minutes_played):

    p90_value = round((((variable_value/data_fix['MoP']))*90),2)
    
    return p90_value

p90 = data_fix.apply(p90_Calculator, args = [1])

#########
p90['Name'] = df_data['Name']
p90['Team'] = df_data['Team']
p90['MoP'] = df_data['MoP']

#ontarget
def ontarget_ratio(row):
  ratio = 0
  if row['Shot on'] > 0:
    ratio = round(((row['Shot on']/row['Shots'])*100),2)
  
  return ratio

p90['Shot on Target Ratio'] = data_fix.apply(lambda row: ontarget_ratio(row), axis=1)

#passacc
def pass_acc(row):
  ratio = 0
  if row['Pass'] > 0:
    ratio = round(((row['Pass']/row['Passes'])*100),2)
  
  return ratio

p90['Pass Accuracy'] = data_fix.apply(lambda row: pass_acc(row), axis=1)

#conversion
def conv_ratio(row):
  ratio = 0
  if row['Goals'] > 0:
    ratio = round(((row['Goals']/row['Shots'])*100),2)
  
  return ratio

p90['Conversion Ratio'] = data_fix.apply(lambda row: conv_ratio(row), axis=1)

#aerial won
def ae_won(row):
  ratio = 0
  if row['Aerial Won'] > 0:
    ratio = round(((row['Aerial Won']/row['Aerial Duels'])*100),2)
  
  return ratio

p90['Aerial Won %'] = data_fix.apply(lambda row: ae_won(row), axis=1)

#saves
def save_per(row):
  ratio = 0
  if row['Saves'] > 0 and row['Shots on Target Faced'] > 0:
    ratio = round(((row['Saves']/row['Shots on Target Faced'])*100),2)
  
  return ratio

p90['Save Percentage'] = data_fix.apply(lambda row: save_per(row), axis=1)

#lgk
def lgk(row):
  ratio = 0
  if row['Goal Kick - Goal Kick Launch'] > 0:
    ratio = round(((row['Goal Kick - Goal Kick Launch']/row['Goal Kick'])*100),2)
  
  return ratio

p90['Long Goal Kick %'] = data_fix.apply(lambda row: lgk(row), axis=1)

pos = player_db[['Name','Position']]

data_full = pd.merge(p90,pos,on='Name',how='left')

#########
data_full = data_full.loc[(data_full['MoP']>179)]

df4 = data_full.groupby('Position', as_index=False)

midfielder = df4.get_group('Midfielder')
goalkeeper = df4.get_group('Goalkeeper')
forward = df4.get_group('Forward')
att_10 = df4.get_group('Attacking 10')
center_back = df4.get_group('Center Back')
fullback = df4.get_group('Fullback')
winger = df4.get_group('Winger')

#winger
winger.loc['mean'] = round((winger.mean()),2)
values1 = {"Name": 'Average W', "Position": 'Winger', "Team": 'League Average'}
winger = winger.fillna(value=values1)

#fb
fullback.loc['mean'] = round((fullback.mean()),2)
values2 = {"Name": 'Average FB', "Position": 'Fullback', "Team": 'League Average'}
fullback = fullback.fillna(value=values2)

#cb
center_back.loc['mean'] = round((center_back.mean()),2)
values3 = {"Name": 'Average CB', "Position": 'Center Back', "Team": 'League Average'}
center_back = center_back.fillna(value=values3)

#cam
att_10.loc['mean'] = round((att_10.mean()),2)
values4 = {"Name": 'Average CAM', "Position": 'Attacking 10', "Team": 'League Average'}
att_10 = att_10.fillna(value=values4)

#forward
forward.loc['mean'] = round((forward.mean()),2)
values5 = {"Name": 'Average FW', "Position": 'Forward', "Team": 'League Average'}
forward = forward.fillna(value=values5)

#gk
goalkeeper.loc['mean'] = round((goalkeeper.mean()),2)
values6 = {"Name": 'Average GK', "Position": 'Goalkeeper', "Team": 'League Average'}
goalkeeper = goalkeeper.fillna(value=values6)

#cm
midfielder.loc['mean'] = round((midfielder.mean()),2)
values7 = {"Name": 'Average CM', "Position": 'Midfielder', "Team": 'League Average'}
midfielder = midfielder.fillna(value=values7)


rank_cm = round(((midfielder.rank(pct=True))*100),2)
rank_gk = round(((goalkeeper.rank(pct=True))*100),2)
rank_fw = round(((forward.rank(pct=True))*100),2)
rank_cam = round(((att_10.rank(pct=True))*100),2)
rank_cb = round(((center_back.rank(pct=True))*100),2)
rank_fb = round(((fullback.rank(pct=True))*100),2)
rank_w = round(((winger.rank(pct=True))*100),2)

rank_cm['Name'] = midfielder['Name']
rank_gk['Name'] = goalkeeper['Name']
rank_fw['Name'] = forward['Name']
rank_cam['Name'] = att_10['Name']
rank_cb['Name'] = center_back['Name']
rank_fb['Name'] = fullback['Name']
rank_w['Name'] = winger['Name']

rank_cm['Team'] = midfielder['Team']
rank_gk['Team'] = goalkeeper['Team']
rank_fw['Team'] = forward['Team']
rank_cam['Team'] = att_10['Team']
rank_cb['Team'] = center_back['Team']
rank_fb['Team'] = fullback['Team']
rank_w['Team'] = winger['Team']


rank_cm['Position'] = midfielder['Position']
rank_gk['Position'] = goalkeeper['Position']
rank_fw['Position'] = forward['Position']
rank_cam['Position'] = att_10['Position']
rank_cb['Position'] = center_back['Position']
rank_fb['Position'] = fullback['Position']
rank_w['Position'] = winger['Position']

rank_liga2 = pd.concat([rank_cm, rank_gk, rank_fw, rank_cam, rank_cb, rank_fb, rank_w])

rank_liga2 = rank_liga2.drop(['MoP'],axis=1)
rank_liga2 = rank_liga2.add_suffix('_pct')
rank_liga2 = rank_liga2.rename(columns={'Name_pct': 'Name'})

from datetime import date
date = date.today().strftime("%Y-%m-%d")
filename = 'pct_rank_liga2_' + date +'.xlsx'

rank_liga2.to_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Percentile/Liga 2/'+filename, index=False)

In [ ]:
df3 = df2[df2['Kompetisi'].str.contains("Liga 1")]
df3 = df3.drop(['No','Gameweek','Opponent','Position (in match)','Match','Home/Away','Venue','Date','Result','Starter/Subs','Subs','Ball Possession','DoB','Kompetisi','Position','Nationality','Pref. Foot','badge'], axis=1)

df_sum = df3.groupby(['Name','Team'], as_index=False).sum()

df_data = df_sum[['Name','Team','MoP','Goal','Shot on','Shot off','Shot Blocked','Create Chance','Assist','Pass','Pass Fail','Cross','Dribble','Tackle','Intercept','Intercept Fail','Recovery','Block','Aerial Won','Aerial Lost','Save','Shots on Target Faced','Penalty Save','Concede Penalty','Keeper - Sweeper', 'Cross Claim','Error Goal', 'Goal Kick', 'Goal Kick - Goal Kick Launch', 'Penalty Goal', 'Pass - Progressive Pass', 'Pass - Through Pass']]

df_data['Shots'] = df_data['Shot on'] + df_data['Shot off'] + df_data['Shot Blocked']
df_data['Aerial Duels'] = df_data['Aerial Won'] + df_data['Aerial Lost']
df_data['Passes'] = df_data['Pass'] + df_data['Pass Fail']
df_data['Intercepts'] = df_data['Intercept'] + df_data['Intercept Fail']
df_data['Saves'] = df_data['Save'] + df_data['Penalty Save']
df_data['Goals'] = df_data['Goal'] + df_data['Penalty Goal']

##########
data_fix = df_data[['Name','Team','MoP','Goals','Shots','Shot on','Create Chance','Assist','Pass','Passes','Pass - Progressive Pass', 'Pass - Through Pass','Cross','Dribble','Tackle','Intercepts','Recovery','Block','Aerial Duels','Aerial Won','Saves','Shots on Target Faced','Penalty Save','Keeper - Sweeper','Cross Claim','Goal Kick','Goal Kick - Goal Kick Launch']]

data_fix = data_fix.drop(['Name','Team'], axis=1)

def p90_Calculator(variable_value, minutes_played):

    p90_value = round((((variable_value/data_fix['MoP']))*90),2)
    
    return p90_value

p90 = data_fix.apply(p90_Calculator, args = [1])

#########
p90['Name'] = df_data['Name']
p90['Team'] = df_data['Team']
p90['MoP'] = df_data['MoP']

#ontarget
def ontarget_ratio(row):
  ratio = 0
  if row['Shot on'] > 0:
    ratio = round(((row['Shot on']/row['Shots'])*100),2)
  
  return ratio

p90['Shot on Target Ratio'] = data_fix.apply(lambda row: ontarget_ratio(row), axis=1)

#passacc
def pass_acc(row):
  ratio = 0
  if row['Pass'] > 0:
    ratio = round(((row['Pass']/row['Passes'])*100),2)
  
  return ratio

p90['Pass Accuracy'] = data_fix.apply(lambda row: pass_acc(row), axis=1)

#conversion
def conv_ratio(row):
  ratio = 0
  if row['Goals'] > 0:
    ratio = round(((row['Goals']/row['Shots'])*100),2)
  
  return ratio

p90['Conversion Ratio'] = data_fix.apply(lambda row: conv_ratio(row), axis=1)

#aerial won
def ae_won(row):
  ratio = 0
  if row['Aerial Won'] > 0:
    ratio = round(((row['Aerial Won']/row['Aerial Duels'])*100),2)
  
  return ratio

p90['Aerial Won %'] = data_fix.apply(lambda row: ae_won(row), axis=1)

#saves
def save_per(row):
  ratio = 0
  if row['Saves'] > 0 and row['Shots on Target Faced'] > 0:
    ratio = round(((row['Saves']/row['Shots on Target Faced'])*100),2)
  
  return ratio

p90['Save Percentage'] = data_fix.apply(lambda row: save_per(row), axis=1)

#lgk
def lgk(row):
  ratio = 0
  if row['Goal Kick - Goal Kick Launch'] > 0:
    ratio = round(((row['Goal Kick - Goal Kick Launch']/row['Goal Kick'])*100),2)
  
  return ratio

p90['Long Goal Kick %'] = data_fix.apply(lambda row: lgk(row), axis=1)

pos = player_db[['Name','Position']]

data_full = pd.merge(p90,pos,on='Name',how='left')

#########
data_full = data_full.loc[(data_full['MoP']>269)]

df4 = data_full.groupby('Position', as_index=False)

midfielder = df4.get_group('Midfielder')
goalkeeper = df4.get_group('Goalkeeper')
forward = df4.get_group('Forward')
att_10 = df4.get_group('Attacking 10')
center_back = df4.get_group('Center Back')
fullback = df4.get_group('Fullback')
winger = df4.get_group('Winger')

#winger
winger.loc['mean'] = round((winger.mean()),2)
values1 = {"Name": 'Average W', "Position": 'Winger', "Team": 'League Average'}
winger = winger.fillna(value=values1)

#fb
fullback.loc['mean'] = round((fullback.mean()),2)
values2 = {"Name": 'Average FB', "Position": 'Fullback', "Team": 'League Average'}
fullback = fullback.fillna(value=values2)

#cb
center_back.loc['mean'] = round((center_back.mean()),2)
values3 = {"Name": 'Average CB', "Position": 'Center Back', "Team": 'League Average'}
center_back = center_back.fillna(value=values3)

#cam
att_10.loc['mean'] = round((att_10.mean()),2)
values4 = {"Name": 'Average CAM', "Position": 'Attacking 10', "Team": 'League Average'}
att_10 = att_10.fillna(value=values4)

#forward
forward.loc['mean'] = round((forward.mean()),2)
values5 = {"Name": 'Average FW', "Position": 'Forward', "Team": 'League Average'}
forward = forward.fillna(value=values5)

#gk
goalkeeper.loc['mean'] = round((goalkeeper.mean()),2)
values6 = {"Name": 'Average GK', "Position": 'Goalkeeper', "Team": 'League Average'}
goalkeeper = goalkeeper.fillna(value=values6)

#cm
midfielder.loc['mean'] = round((midfielder.mean()),2)
values7 = {"Name": 'Average CM', "Position": 'Midfielder', "Team": 'League Average'}
midfielder = midfielder.fillna(value=values7)


rank_cm = round(((midfielder.rank(pct=True))*100),2)
rank_gk = round(((goalkeeper.rank(pct=True))*100),2)
rank_fw = round(((forward.rank(pct=True))*100),2)
rank_cam = round(((att_10.rank(pct=True))*100),2)
rank_cb = round(((center_back.rank(pct=True))*100),2)
rank_fb = round(((fullback.rank(pct=True))*100),2)
rank_w = round(((winger.rank(pct=True))*100),2)

rank_cm['Name'] = midfielder['Name']
rank_gk['Name'] = goalkeeper['Name']
rank_fw['Name'] = forward['Name']
rank_cam['Name'] = att_10['Name']
rank_cb['Name'] = center_back['Name']
rank_fb['Name'] = fullback['Name']
rank_w['Name'] = winger['Name']

rank_cm['Team'] = midfielder['Team']
rank_gk['Team'] = goalkeeper['Team']
rank_fw['Team'] = forward['Team']
rank_cam['Team'] = att_10['Team']
rank_cb['Team'] = center_back['Team']
rank_fb['Team'] = fullback['Team']
rank_w['Team'] = winger['Team']

rank_cm['Position'] = midfielder['Position']
rank_gk['Position'] = goalkeeper['Position']
rank_fw['Position'] = forward['Position']
rank_cam['Position'] = att_10['Position']
rank_cb['Position'] = center_back['Position']
rank_fb['Position'] = fullback['Position']
rank_w['Position'] = winger['Position']

rank_liga1 = pd.concat([rank_cm, rank_gk, rank_fw, rank_cam, rank_cb, rank_fb, rank_w])

rank_liga1 = rank_liga1.drop(['MoP'],axis=1)
rank_liga1 = rank_liga1.add_suffix('_pct')
rank_liga1 = rank_liga1.rename(columns={'Name_pct': 'Name'})

from datetime import date
date = date.today().strftime("%Y-%m-%d")
filename = 'pct_rank_liga1_' + date +'.xlsx'
filename2 = 'pct_rank_' + date +'.xlsx'

rank_liga1.to_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Percentile/Liga 1/'+filename, index=False)

rank = pd.concat([rank_liga1,rank_liga2])

rank.to_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Percentile/'+filename2, index=False)

In [9]:
from datetime import date

df2['DoB'] = pd.to_datetime(df2.DoB)

today = date.today()
df2['Age'] = df2['DoB'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))

df2['DoB'] = df2['DoB'].dt.strftime('%d/%m/%Y')

def age_group(row):  
    if row['Age'] < 21:
        return 'U21'
    elif 21 <= row['Age'] <= 32:
        return 'Prime'
    elif row['Age'] > 32:
        return 'Senior'

df2['Age Group'] = df2.apply(lambda row: age_group(row), axis=1)

def status(row):  
    if row['Nationality'] == 'Indonesia':
        return 'Local'
    elif row['Nationality'] != 'Indonesia':
        return 'Foreign'

df2['Status'] = df2.apply(lambda row: status(row), axis=1)

rank = rank.drop(['Position_pct','Team_pct'],axis=1)

df_fin = pd.merge(df2,rank,on='Name',how='left')

date = date.today().strftime("%Y-%m-%d")
filename = 'dashboard_upd' + date + '_3' + '.xlsx'

df_fin.to_excel('/content/gdrive/MyDrive/Liga Indonesia 2022/Dashboard/'+filename, index=False)

In [ ]:
df_fin.head()